In [1]:
import tensorflow as tf
import numpy as np


In [2]:
def read_data(path):
    filename = tf.matching_files(path)
    filename_queue = tf.train.string_input_producer(filename)
    reader = tf.TFRecordReader()
    _, example = reader.read(filename_queue)
    features = tf.parse_single_example(
        example,
        features={
            'image_raw': tf.FixedLenFeature([], tf.string),
            'label': tf.FixedLenFeature([], tf.int64),
        })

    images = tf.decode_raw(features['image_raw'], tf.uint8)
    images = tf.reshape(images, [1728])
    labels = tf.cast(features['label'], tf.int64)
    return images / 255, labels

In [3]:
# 将图片shape转化为(?, 4, 10 )
def image_reshape(l):
    lab = np.zeros([len(l), 4, 11])
    for n in range(len(l)):
        for j, j_ in enumerate(str(l[n]) + ' ' * (4 - len(str(l[n])))):
            if j_ == ' ':
                lab[n][j][10] = 1
            else:
                lab[n][j][int(j_)] = 1
    return lab

In [4]:
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.1))


def init_bias(shape):
    return tf.Variable(tf.constant(0.1, shape=shape))

In [5]:
x_ = tf.placeholder(tf.float32, [None, 1728])
y = tf.placeholder(tf.float32, [None, 4, 11])
keep_prob = tf.placeholder(tf.float32, name='keep_prob')
x = tf.reshape(x_, [-1, 36, 48, 1], name='x')

# 第一层卷积
w1 = init_weights([3, 3, 1, 32])
b1 = init_bias([32])
conv1 = tf.nn.relu(tf.nn.conv2d(x, w1, strides=[1, 1, 1, 1], padding='SAME') + b1)  # shape=(?, 36, 48, 32)
# 池化
pool1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')  # # shape=(?, 18, 24, 32)

# 第二层卷积
w2 = init_weights([3, 3, 32, 64])
b2 = init_bias([64])
conv2 = tf.nn.relu(tf.nn.conv2d(pool1, w2, strides=[1, 1, 1, 1], padding='SAME') + b2)  # shape=(?, 18, 24, 64)
# 池化
pool2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')  # shape=(?, 9, 12, 64)

# 第三层卷积
w3 = init_weights([3, 3, 64, 96])
b3 = init_bias([96])
conv3 = tf.nn.relu(tf.nn.conv2d(pool2, w3, strides=[1, 1, 1, 1], padding='SAME') + b3)  # shape=(?, 9, 12, 96)
# 池化
pool3_ = tf.nn.max_pool(conv3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')  # shape=(?, 5, 6, 96)

pool3 = tf.reshape(pool3_, [-1, 5 * 6 * 96])

# 全连接层
w4 = init_weights([5 * 6 * 96, 512])
b4 = init_bias([512])
h = tf.nn.relu(tf.matmul(pool3, w4) + b4)
h = tf.nn.dropout(h, keep_prob)

# 输出层 1
w_o = init_weights([512, 11])
b_o = init_bias([11])
y_o = tf.matmul(h, w_o) + b_o


# 输出层 2
w_o1 = init_weights([512, 11])
b_o1 = init_bias([11])
y_o1 = tf.matmul(h, w_o1) + b_o1


# 输出层 3
w_o2 = init_weights([512, 11])
b_o2 = init_bias([11])
y_o2 = tf.matmul(h, w_o2) + b_o2

# 输出层 4
w_o3 = init_weights([512, 11])
b_o3 = init_bias([11])
y_o3 = tf.matmul(h, w_o3) + b_o3

y_oo = tf.concat([y_o, y_o1, y_o2, y_o3], 1)
y_oo = tf.reshape(y_oo, [-1, 4, 11])

cost1 = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_oo[:, 0], labels=y[:, 0]))
cost2 = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_oo[:, 1], labels=y[:, 1]))
cost3 = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_oo[:, 2], labels=y[:, 2]))
cost4 = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_oo[:, 3], labels=y[:, 3]))
# cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_oo, labels=y))
cost = tf.reduce_sum([cost1, cost2, cost3, cost4])
train_op = tf.train.AdamOptimizer().minimize(cost)
prediction = tf.equal(tf.argmax(y_oo, 2), tf.argmax(y, 2))
accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32), name='accuracy')


t_accuracy = 0
batch_size = 1

In [ ]:
with tf.Session() as sess:
    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())
    image, label = read_data('./tfrecord/test*')
    images, labels = tf.train.shuffle_batch([image, label], batch_size=4000, num_threads=1,
                                            capacity=2000, min_after_dequeue=1000)
    saver.restore(sess, './model/model.ckpt')

    coord = tf.train.Coordinator()
    thread = tf.train.start_queue_runners(coord=coord)
    img, ll = sess.run([images, labels])
    lab = image_reshape(ll)
    test_batch = zip(range(0, len(img), batch_size),
                         range(batch_size, len(img) + 1, batch_size))
    for start, end in test_batch:

        acc, co, a, b = sess.run([accuracy, cost, tf.argmax(y_oo, 2), tf.argmax(y, 2)],
                                            feed_dict={x_: img[start:end], y: lab[start:end], keep_prob: 1.0})

        if acc == 1:
            t_accuracy += 1
        print('是否正确：', acc, '预测值：', a, '正确值：', b)
    print('正确率：', t_accuracy/4000)
    coord.request_stop()
    coord.join(thread)

INFO:tensorflow:Restoring parameters from ./model/model.ckpt
是否正确： 1.0 预测值： [[ 7  0 10 10]] 正确值： [[ 7  0 10 10]]
是否正确： 1.0 预测值： [[ 3  0 10 10]] 正确值： [[ 3  0 10 10]]
是否正确： 1.0 预测值： [[ 6 10 10 10]] 正确值： [[ 6 10 10 10]]
是否正确： 1.0 预测值： [[ 3  5 10 10]] 正确值： [[ 3  5 10 10]]
是否正确： 1.0 预测值： [[ 7  9 10 10]] 正确值： [[ 7  9 10 10]]
是否正确： 1.0 预测值： [[ 3 10 10 10]] 正确值： [[ 3 10 10 10]]
是否正确： 1.0 预测值： [[ 1  8 10 10]] 正确值： [[ 1  8 10 10]]
是否正确： 0.75 预测值： [[ 8  0 10 10]] 正确值： [[ 8  1 10 10]]
是否正确： 0.75 预测值： [[ 1 10 10 10]] 正确值： [[ 7 10 10 10]]
是否正确： 1.0 预测值： [[ 3 10 10 10]] 正确值： [[ 3 10 10 10]]
是否正确： 1.0 预测值： [[ 2 10 10 10]] 正确值： [[ 2 10 10 10]]
是否正确： 1.0 预测值： [[ 6 10 10 10]] 正确值： [[ 6 10 10 10]]
是否正确： 1.0 预测值： [[ 2 10 10 10]] 正确值： [[ 2 10 10 10]]
是否正确： 1.0 预测值： [[ 9 10 10 10]] 正确值： [[ 9 10 10 10]]
是否正确： 1.0 预测值： [[ 6  5 10 10]] 正确值： [[ 6  5 10 10]]
是否正确： 1.0 预测值： [[ 9  3 10 10]] 正确值： [[ 9  3 10 10]]
是否正确： 1.0 预测值： [[ 5 10 10 10]] 正确值： [[ 5 10 10 10]]
是否正确： 1.0 预测值： [[ 4  2 10 10]] 正确值： [[ 4  2 10 10]]
是

是否正确： 1.0 预测值： [[ 4 10 10 10]] 正确值： [[ 4 10 10 10]]
是否正确： 1.0 预测值： [[ 4  4 10 10]] 正确值： [[ 4  4 10 10]]
是否正确： 1.0 预测值： [[ 9  2 10 10]] 正确值： [[ 9  2 10 10]]
是否正确： 1.0 预测值： [[ 9  1 10 10]] 正确值： [[ 9  1 10 10]]
是否正确： 1.0 预测值： [[ 2 10 10 10]] 正确值： [[ 2 10 10 10]]
是否正确： 0.75 预测值： [[ 1  1 10 10]] 正确值： [[ 1  7 10 10]]
是否正确： 1.0 预测值： [[ 1 10 10 10]] 正确值： [[ 1 10 10 10]]
是否正确： 1.0 预测值： [[ 6  4 10 10]] 正确值： [[ 6  4 10 10]]
是否正确： 1.0 预测值： [[ 9  4 10 10]] 正确值： [[ 9  4 10 10]]
是否正确： 1.0 预测值： [[ 3 10 10 10]] 正确值： [[ 3 10 10 10]]
是否正确： 1.0 预测值： [[ 3 10 10 10]] 正确值： [[ 3 10 10 10]]
是否正确： 1.0 预测值： [[ 6  4 10 10]] 正确值： [[ 6  4 10 10]]
是否正确： 1.0 预测值： [[ 6 10 10 10]] 正确值： [[ 6 10 10 10]]
是否正确： 1.0 预测值： [[ 5 10 10 10]] 正确值： [[ 5 10 10 10]]
是否正确： 1.0 预测值： [[ 8 10 10 10]] 正确值： [[ 8 10 10 10]]
是否正确： 1.0 预测值： [[ 5 10 10 10]] 正确值： [[ 5 10 10 10]]
是否正确： 1.0 预测值： [[ 3  1 10 10]] 正确值： [[ 3  1 10 10]]
是否正确： 1.0 预测值： [[ 7 10 10 10]] 正确值： [[ 7 10 10 10]]
是否正确： 1.0 预测值： [[ 5  7 10 10]] 正确值： [[ 5  7 10 10]]
是否正确： 1.0 预

是否正确： 0.75 预测值： [[ 1  9 10 10]] 正确值： [[ 7  9 10 10]]
是否正确： 0.75 预测值： [[ 4  1  1 10]] 正确值： [[ 4  1 10 10]]
是否正确： 1.0 预测值： [[ 9  2 10 10]] 正确值： [[ 9  2 10 10]]
是否正确： 1.0 预测值： [[ 4 10 10 10]] 正确值： [[ 4 10 10 10]]
是否正确： 1.0 预测值： [[ 5 10 10 10]] 正确值： [[ 5 10 10 10]]
是否正确： 1.0 预测值： [[ 6 10 10 10]] 正确值： [[ 6 10 10 10]]
是否正确： 1.0 预测值： [[ 4  9 10 10]] 正确值： [[ 4  9 10 10]]
是否正确： 1.0 预测值： [[ 4 10 10 10]] 正确值： [[ 4 10 10 10]]
是否正确： 1.0 预测值： [[ 9 10 10 10]] 正确值： [[ 9 10 10 10]]
是否正确： 1.0 预测值： [[ 5 10 10 10]] 正确值： [[ 5 10 10 10]]
是否正确： 1.0 预测值： [[ 7 10 10 10]] 正确值： [[ 7 10 10 10]]
是否正确： 1.0 预测值： [[ 6 10 10 10]] 正确值： [[ 6 10 10 10]]
是否正确： 1.0 预测值： [[ 8 10 10 10]] 正确值： [[ 8 10 10 10]]
是否正确： 1.0 预测值： [[ 5  4 10 10]] 正确值： [[ 5  4 10 10]]
是否正确： 1.0 预测值： [[ 5  5 10 10]] 正确值： [[ 5  5 10 10]]
是否正确： 1.0 预测值： [[ 3  0 10 10]] 正确值： [[ 3  0 10 10]]
是否正确： 1.0 预测值： [[ 7  8 10 10]] 正确值： [[ 7  8 10 10]]
是否正确： 1.0 预测值： [[ 9 10 10 10]] 正确值： [[ 9 10 10 10]]
是否正确： 1.0 预测值： [[ 2  1 10 10]] 正确值： [[ 2  1 10 10]]
是否正确： 0.75

是否正确： 1.0 预测值： [[ 3 10 10 10]] 正确值： [[ 3 10 10 10]]
是否正确： 1.0 预测值： [[ 3 10 10 10]] 正确值： [[ 3 10 10 10]]
是否正确： 1.0 预测值： [[ 8  7 10 10]] 正确值： [[ 8  7 10 10]]
是否正确： 1.0 预测值： [[ 2  6 10 10]] 正确值： [[ 2  6 10 10]]
是否正确： 1.0 预测值： [[ 8 10 10 10]] 正确值： [[ 8 10 10 10]]
是否正确： 1.0 预测值： [[ 9  1 10 10]] 正确值： [[ 9  1 10 10]]
是否正确： 1.0 预测值： [[ 5 10 10 10]] 正确值： [[ 5 10 10 10]]
是否正确： 1.0 预测值： [[ 9 10 10 10]] 正确值： [[ 9 10 10 10]]
是否正确： 1.0 预测值： [[ 8 10 10 10]] 正确值： [[ 8 10 10 10]]
是否正确： 1.0 预测值： [[ 2 10 10 10]] 正确值： [[ 2 10 10 10]]
是否正确： 1.0 预测值： [[ 1  9 10 10]] 正确值： [[ 1  9 10 10]]
是否正确： 1.0 预测值： [[ 3  5 10 10]] 正确值： [[ 3  5 10 10]]
是否正确： 1.0 预测值： [[ 1 10 10 10]] 正确值： [[ 1 10 10 10]]
是否正确： 1.0 预测值： [[ 1 10 10 10]] 正确值： [[ 1 10 10 10]]
是否正确： 1.0 预测值： [[ 9  5 10 10]] 正确值： [[ 9  5 10 10]]
是否正确： 1.0 预测值： [[ 1 10 10 10]] 正确值： [[ 1 10 10 10]]
是否正确： 1.0 预测值： [[ 7 10 10 10]] 正确值： [[ 7 10 10 10]]
是否正确： 1.0 预测值： [[ 1 10 10 10]] 正确值： [[ 1 10 10 10]]
是否正确： 1.0 预测值： [[ 2 10 10 10]] 正确值： [[ 2 10 10 10]]
是否正确： 0.75 预

是否正确： 1.0 预测值： [[ 6 10 10 10]] 正确值： [[ 6 10 10 10]]
是否正确： 1.0 预测值： [[ 1  3 10 10]] 正确值： [[ 1  3 10 10]]
是否正确： 1.0 预测值： [[ 5 10 10 10]] 正确值： [[ 5 10 10 10]]
是否正确： 1.0 预测值： [[ 3 10 10 10]] 正确值： [[ 3 10 10 10]]
是否正确： 1.0 预测值： [[ 3 10 10 10]] 正确值： [[ 3 10 10 10]]
是否正确： 1.0 预测值： [[ 4 10 10 10]] 正确值： [[ 4 10 10 10]]
是否正确： 1.0 预测值： [[ 9 10 10 10]] 正确值： [[ 9 10 10 10]]
是否正确： 1.0 预测值： [[ 8 10 10 10]] 正确值： [[ 8 10 10 10]]
是否正确： 1.0 预测值： [[ 7  2 10 10]] 正确值： [[ 7  2 10 10]]
是否正确： 1.0 预测值： [[ 6 10 10 10]] 正确值： [[ 6 10 10 10]]
是否正确： 1.0 预测值： [[ 7  2 10 10]] 正确值： [[ 7  2 10 10]]
是否正确： 1.0 预测值： [[ 6  4 10 10]] 正确值： [[ 6  4 10 10]]
是否正确： 0.75 预测值： [[ 4 10 10 10]] 正确值： [[ 7 10 10 10]]
是否正确： 0.75 预测值： [[ 1  2 10 10]] 正确值： [[ 7  2 10 10]]
是否正确： 1.0 预测值： [[ 4  4 10 10]] 正确值： [[ 4  4 10 10]]
是否正确： 1.0 预测值： [[ 7  0 10 10]] 正确值： [[ 7  0 10 10]]
是否正确： 1.0 预测值： [[ 4 10 10 10]] 正确值： [[ 4 10 10 10]]
是否正确： 1.0 预测值： [[ 2  8 10 10]] 正确值： [[ 2  8 10 10]]
是否正确： 1.0 预测值： [[ 9  4 10 10]] 正确值： [[ 9  4 10 10]]
是否正确： 1.0 

...

是否正确： 1.0 预测值： [[ 3  2  0 10]] 正确值： [[ 3  2  0 10]]
是否正确： 0.75 预测值： [[ 2  2  2 10]] 正确值： [[ 2  8  2 10]]
是否正确： 1.0 预测值： [[3 1 3 7]] 正确值： [[3 1 3 7]]
是否正确： 0.75 预测值： [[ 5  6  9 10]] 正确值： [[ 5  0  9 10]]
是否正确： 1.0 预测值： [[ 5 10 10 10]] 正确值： [[ 5 10 10 10]]
是否正确： 1.0 预测值： [[ 9  3 10 10]] 正确值： [[ 9  3 10 10]]
是否正确： 1.0 预测值： [[8 9 4 4]] 正确值： [[8 9 4 4]]
是否正确： 1.0 预测值： [[ 7  1 10 10]] 正确值： [[ 7  1 10 10]]
是否正确： 1.0 预测值： [[ 7  2  4 10]] 正确值： [[ 7  2  4 10]]
是否正确： 0.75 预测值： [[ 8  8  0 10]] 正确值： [[ 8  8  6 10]]
是否正确： 1.0 预测值： [[ 6  3  4 10]] 正确值： [[ 6  3  4 10]]
是否正确： 1.0 预测值： [[ 7  3  8 10]] 正确值： [[ 7  3  8 10]]
是否正确： 1.0 预测值： [[ 3  0  8 10]] 正确值： [[ 3  0  8 10]]
是否正确： 1.0 预测值： [[ 6  5  4 10]] 正确值： [[ 6  5  4 10]]
是否正确： 1.0 预测值： [[ 2  5  3 10]] 正确值： [[ 2  5  3 10]]
是否正确： 1.0 预测值： [[3 0 8 2]] 正确值： [[3 0 8 2]]
是否正确： 1.0 预测值： [[ 4  5  0 10]] 正确值： [[ 4  5  0 10]]
是否正确： 1.0 预测值： [[9 1 9 9]] 正确值： [[9 1 9 9]]
是否正确： 1.0 预测值： [[ 3  8  5 10]] 正确值： [[ 3  8  5 10]]
是否正确： 1.0 预测值： [[ 6  6  4 10]] 正确值： [[ 6 

是否正确： 0.75 预测值： [[ 2  3  7 10]] 正确值： [[ 2  6  7 10]]
是否正确： 0.75 预测值： [[ 2  8  6 10]] 正确值： [[ 2  8  5 10]]
是否正确： 0.75 预测值： [[ 2  6  3 10]] 正确值： [[ 7  6  3 10]]
是否正确： 1.0 预测值： [[ 5  4  7 10]] 正确值： [[ 5  4  7 10]]
是否正确： 1.0 预测值： [[5 7 4 0]] 正确值： [[5 7 4 0]]
是否正确： 0.75 预测值： [[ 9  0  6 10]] 正确值： [[ 3  0  6 10]]
是否正确： 1.0 预测值： [[ 9  3  8 10]] 正确值： [[ 9  3  8 10]]
是否正确： 1.0 预测值： [[ 8  0  0 10]] 正确值： [[ 8  0  0 10]]
是否正确： 1.0 预测值： [[ 7  6  5 10]] 正确值： [[ 7  6  5 10]]
是否正确： 0.75 预测值： [[ 2  0  2 10]] 正确值： [[ 2  1  2 10]]
是否正确： 1.0 预测值： [[ 4  6  9 10]] 正确值： [[ 4  6  9 10]]
是否正确： 0.75 预测值： [[7 7 0 3]] 正确值： [[7 7 9 3]]
是否正确： 1.0 预测值： [[ 8  5  5 10]] 正确值： [[ 8  5  5 10]]
是否正确： 1.0 预测值： [[7 7 0 4]] 正确值： [[7 7 0 4]]
是否正确： 1.0 预测值： [[ 8 10 10 10]] 正确值： [[ 8 10 10 10]]
是否正确： 1.0 预测值： [[ 9 10 10 10]] 正确值： [[ 9 10 10 10]]
是否正确： 0.75 预测值： [[ 2  1  7 10]] 正确值： [[ 2  1  1 10]]
是否正确： 1.0 预测值： [[ 5  4 10 10]] 正确值： [[ 5  4 10 10]]
是否正确： 1.0 预测值： [[ 1  3  9 10]] 正确值： [[ 1  3  9 10]]
是否正确： 1.0 预测值： [[ 1  5  7 10]

是否正确： 0.75 预测值： [[ 5  0  0 10]] 正确值： [[ 5  9  0 10]]
是否正确： 1.0 预测值： [[ 4  2  7 10]] 正确值： [[ 4  2  7 10]]
是否正确： 1.0 预测值： [[ 8  2  0 10]] 正确值： [[ 8  2  0 10]]
是否正确： 1.0 预测值： [[ 2  0  9 10]] 正确值： [[ 2  0  9 10]]
是否正确： 1.0 预测值： [[ 2  8  9 10]] 正确值： [[ 2  8  9 10]]
是否正确： 0.75 预测值： [[ 2  8  7 10]] 正确值： [[ 2  8  1 10]]
是否正确： 0.75 预测值： [[ 1  3  1 10]] 正确值： [[ 7  3  1 10]]
是否正确： 1.0 预测值： [[ 2  2  2 10]] 正确值： [[ 2  2  2 10]]
是否正确： 1.0 预测值： [[ 9  6 10 10]] 正确值： [[ 9  6 10 10]]
是否正确： 1.0 预测值： [[ 2  6  6 10]] 正确值： [[ 2  6  6 10]]
是否正确： 1.0 预测值： [[ 2  6 10 10]] 正确值： [[ 2  6 10 10]]
是否正确： 0.75 预测值： [[ 1  5  7 10]] 正确值： [[ 4  5  7 10]]
是否正确： 0.5 预测值： [[ 5  8  0 10]] 正确值： [[5 8 4 7]]
是否正确： 1.0 预测值： [[ 7 10 10 10]] 正确值： [[ 7 10 10 10]]
是否正确： 0.75 预测值： [[2 8 7 3]] 正确值： [[2 8 2 3]]
是否正确： 0.75 预测值： [[ 1  5  8 10]] 正确值： [[ 7  5  8 10]]
是否正确： 0.75 预测值： [[6 9 0 1]] 正确值： [[5 9 0 1]]
是否正确： 0.75 预测值： [[ 6  3  1 10]] 正确值： [[ 6  3  4 10]]
是否正确： 1.0 预测值： [[ 5  1  8 10]] 正确值： [[ 5  1  8 10]]
是否正确： 1.0 预测值： [[ 7  0  

是否正确： 0.75 预测值： [[ 2  0  1 10]] 正确值： [[ 2  0  7 10]]
是否正确： 1.0 预测值： [[ 9  0  9 10]] 正确值： [[ 9  0  9 10]]
是否正确： 0.75 预测值： [[ 9  8  9 10]] 正确值： [[ 9  8  7 10]]
是否正确： 1.0 预测值： [[ 4  1  1 10]] 正确值： [[ 4  1  1 10]]
是否正确： 1.0 预测值： [[ 2  6  7 10]] 正确值： [[ 2  6  7 10]]
是否正确： 1.0 预测值： [[6 4 3 3]] 正确值： [[6 4 3 3]]
是否正确： 1.0 预测值： [[ 4  8  7 10]] 正确值： [[ 4  8  7 10]]
是否正确： 1.0 预测值： [[1 0 0 4]] 正确值： [[1 0 0 4]]
是否正确： 0.75 预测值： [[ 1  3  4 10]] 正确值： [[ 4  3  4 10]]
是否正确： 1.0 预测值： [[ 7  9  5 10]] 正确值： [[ 7  9  5 10]]
是否正确： 1.0 预测值： [[ 9  5  8 10]] 正确值： [[ 9  5  8 10]]
是否正确： 1.0 预测值： [[ 6  3  4 10]] 正确值： [[ 6  3  4 10]]
是否正确： 1.0 预测值： [[ 3  8  6 10]] 正确值： [[ 3  8  6 10]]
是否正确： 1.0 预测值： [[ 1  2  6 10]] 正确值： [[ 1  2  6 10]]
是否正确： 0.5 预测值： [[ 2  2  7 10]] 正确值： [[ 2  7  0 10]]
是否正确： 1.0 预测值： [[ 8  7  4 10]] 正确值： [[ 8  7  4 10]]
是否正确： 0.25 预测值： [[2 6 4 4]] 正确值： [[ 7  8  4 10]]
是否正确： 1.0 预测值： [[ 3  7  6 10]] 正确值： [[ 3  7  6 10]]
是否正确： 0.75 预测值： [[ 9  5  0 10]] 正确值： [[ 9  6  0 10]]
是否正确： 1.0 预测值： [[ 7  1  0 1

是否正确： 0.75 预测值： [[ 2  5  5 10]] 正确值： [[ 2  5  8 10]]
是否正确： 1.0 预测值： [[ 8  0  8 10]] 正确值： [[ 8  0  8 10]]
是否正确： 1.0 预测值： [[ 4  5  1 10]] 正确值： [[ 4  5  1 10]]
是否正确： 0.75 预测值： [[ 2  8  2 10]] 正确值： [[ 2  6  2 10]]
是否正确： 1.0 预测值： [[ 4  6  0 10]] 正确值： [[ 4  6  0 10]]
是否正确： 1.0 预测值： [[3 9 3 7]] 正确值： [[3 9 3 7]]
是否正确： 1.0 预测值： [[ 9  6  0 10]] 正确值： [[ 9  6  0 10]]
是否正确： 1.0 预测值： [[ 3  0  8 10]] 正确值： [[ 3  0  8 10]]
是否正确： 1.0 预测值： [[ 5  7  4 10]] 正确值： [[ 5  7  4 10]]
是否正确： 1.0 预测值： [[ 3  0  0 10]] 正确值： [[ 3  0  0 10]]
是否正确： 1.0 预测值： [[5 6 4 6]] 正确值： [[5 6 4 6]]
是否正确： 0.5 预测值： [[ 1  1  9 10]] 正确值： [[ 1  0  1 10]]
是否正确： 1.0 预测值： [[ 8  6  7 10]] 正确值： [[ 8  6  7 10]]
是否正确： 1.0 预测值： [[ 7  5  8 10]] 正确值： [[ 7  5  8 10]]
是否正确： 1.0 预测值： [[ 3  6  5 10]] 正确值： [[ 3  6  5 10]]
是否正确： 0.25 预测值： [[1 2 1 5]] 正确值： [[ 1  7  5 10]]
是否正确： 1.0 预测值： [[ 3  4  3 10]] 正确值： [[ 3  4  3 10]]
是否正确： 0.75 预测值： [[6 8 1 1]] 正确值： [[ 6  8  1 10]]
是否正确： 1.0 预测值： [[ 2  0  6 10]] 正确值： [[ 2  0  6 10]]
是否正确： 1.0 预测值： [[5 1 0 4]] 正确值： 